In [ ]:
import sqlite3
import pandas as pd 
from langchain.prompts import FewShotPromptTemplate,PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
#from langchain.schema import HummanMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import os
import matplotlib.pyplot as plt



### Lets Create a Dummy Dataset

In [5]:
conn=sqlite3.connect("my_data.db")
cursor=conn.cursor()

In [6]:
# Define table schemas
schema_statements = {
    "alarm_data": """
    CREATE TABLE IF NOT EXISTS alarm_data (
        alarm_id TEXT, site_id TEXT, alarm_type TEXT, severity TEXT, timestamp TEXT
    );
    """,
    "incident_data": """
    CREATE TABLE IF NOT EXISTS incident_data (
        incident_id TEXT, site_id TEXT, incident_type TEXT, status TEXT, opened_at TEXT, closed_at TEXT
    );
    """,
    "work_order": """
    CREATE TABLE IF NOT EXISTS work_order (
        wo_id TEXT, site_id TEXT, work_type TEXT, status TEXT, created_at TEXT, closed_at TEXT
    );
    """,
    "network_traffic": """
    CREATE TABLE IF NOT EXISTS network_traffic (
        site_id TEXT, date TEXT, voice_traffic INTEGER, data_traffic INTEGER
    );
    """,
    "network_availability": """
    CREATE TABLE IF NOT EXISTS network_availability (
        site_id TEXT, date TEXT, availability_percent REAL
    );
    """,
    "kpi_summary": """
    CREATE TABLE IF NOT EXISTS kpi_summary (
        date TEXT, total_alarms INTEGER, total_incidents INTEGER, total_wos INTEGER, avg_availability REAL
    );
    """
}
 

In [7]:
# Execute Schema Creation
for smt in schema_statements.values():
    cursor.execute(smt)

In [11]:
# Insert Sample data
cursor.executescript("""
-- Clear previous data if any 
DELETE FROM alarm_data;
DELETE FROM incident_data;
DELETE FROM work_order;
DELETE FROM network_traffic;
DELETE FROM network_availability;
DELETE FROM kpi_summary;

-- Insert alarm_data
INSERT INTO alarm_data VALUES
('A101','S001','Power Failure','Critical','2025-04-01'),
('A102','S001','Link Down','Major','2025-04-02'),
('A103','S002','Signal Loss','Minor','2025-04-03'),
('A104','S003','Overload','Major','2025-04-04'),
('A105','S002','Power Failure','Critical','2025-04-05'),
('A106','S004','Link Down','Warning','2025-04-06'),
('A107','S001','Overload','Major','2025-04-07'),
('A108','S005','Power Failure','Minor','2025-04-08'),
('A109','S003','Link Down','Critical','2025-04-09'),
('A110','S004','Signal Loss','Major','2025-04-10'),
('A111','S005','Overload','Warning','2025-04-11'),
('A112','S002','Power Failure','Critical','2025-04-12'),
('A113','S003','Link Down','Minor','2025-04-13'),
('A114','S004','Signal Loss','Major','2025-04-14'),
('A115','S001','Overload','Critical','2025-04-15'),
('A116','S005','Power Failure','Minor','2025-04-16'),
('A117','S002','Link Down','Major','2025-04-17'),
('A118','S003','Signal Loss','Critical','2025-04-18'),
('A119','S001','Overload','Minor','2025-04-19'),
('A120','S004','Power Failure','Warning','2025-04-20'),
('A118','S003','Signal Loss','Critical','2025-04-21'),
('A119','S001','Overload','Minor','2025-04-22'),
('A120','S004','Power Failure','Warning','2025-04-22');

-- Insert incident_data
INSERT INTO incident_data VALUES
('I201','S001','Link Failure','Closed','2025-04-01','2025-04-01'),
('I202','S002','Power Issue','Open','2025-04-02','2025-04-03'),
('I203','S003','Performance Issue','Closed','2025-04-03','2025-04-04'),
('I204','S004','Security Alert','Closed','2025-04-04','2025-04-05'),
('I205','S005','Link Failure','Open','2025-04-05','2025-04-06'),
('I206','S001','Signal Drop','In Progress','2025-04-06','2025-04-07'),
('I207','S002','Outage','Closed','2025-04-07','2025-04-08'),
('I208','S003','Latency Issue','Open','2025-04-08','2025-04-09'),
('I209','S004','Packet Loss','Closed','2025-04-09','2025-04-10'),
('I210','S005','Routing Error','Closed','2025-04-10','2025-04-11'),
('I211','S001','Firewall Block','Closed','2025-04-11','2025-04-12'),
('I212','S002','System Reboot','Open','2025-04-12','2025-04-13'),
('I213','S003','Network Loop','Closed','2025-04-13','2025-04-14'),
('I214','S004','Hardware Fault','Closed','2025-04-14','2025-04-15'),
('I215','S005','Signal Drop','Closed','2025-04-15','2025-04-16'),
('I216','S001','Data Delay','Open','2025-04-16','2025-04-17'),
('I217','S002','Signal Error','Closed','2025-04-17','2025-04-18'),
('I218','S003','VPN Issue','Closed','2025-04-18','2025-04-19'),
('I219','S004','Firmware Error','Open','2025-04-22','2025-04-20'),
('I220','S005','Access Failure','Closed','2025-04-22','2025-04-21');

-- Insert work_order
INSERT INTO work_order VALUES
('WO101','S001','Repair','Closed','2025-04-01','2025-04-02'),
('WO102','S002','Upgrade','Open','2025-04-02','2025-04-03'),
('WO103','S003','Inspection','Closed','2025-04-03','2025-04-04'),
('WO104','S004','Repair','Closed','2025-04-04','2025-04-05'),
('WO105','S005','Upgrade','Open','2025-04-05','2025-04-06'),
('WO106','S001','Inspection','In Progress','2025-04-06','2025-04-07'),
('WO107','S002','Repair','Closed','2025-04-07','2025-04-08'),
('WO108','S003','Upgrade','Open','2025-04-08','2025-04-09'),
('WO109','S004','Inspection','Closed','2025-04-09','2025-04-10'),
('WO110','S005','Repair','Closed','2025-04-10','2025-04-11'),
('WO111','S001','Upgrade','Closed','2025-04-11','2025-04-12'),
('WO112','S002','Inspection','Open','2025-04-12','2025-04-13'),
('WO113','S003','Repair','Closed','2025-04-13','2025-04-14'),
('WO114','S004','Upgrade','Closed','2025-04-14','2025-04-15'),
('WO115','S005','Inspection','Closed','2025-04-15','2025-04-16'),
('WO116','S001','Repair','Open','2025-04-16','2025-04-17'),
('WO117','S002','Upgrade','Closed','2025-04-17','2025-04-18'),
('WO118','S003','Inspection','Closed','2025-04-22','2025-04-19'),
('WO119','S004','Repair','Open','2025-04-21','2025-04-20'),
('WO120','S005','Upgrade','Closed','2025-04-22','2025-04-21');

-- Insert network_traffic
INSERT INTO network_traffic VALUES
('S001','2025-04-01',120,3000),
('S002','2025-04-02',110,3200),
('S003','2025-04-03',130,2900),
('S004','2025-04-04',125,3100),
('S005','2025-04-05',118,2800),
('S001','2025-04-06',122,3050),
('S002','2025-04-07',111,3150),
('S003','2025-04-08',135,2990),
('S004','2025-04-09',128,3120),
('S005','2025-04-10',117,2820),
('S001','2025-04-11',123,3060),
('S002','2025-04-12',112,3160),
('S003','2025-04-13',136,2980),
('S004','2025-04-14',129,3130),
('S005','2025-04-15',116,2830),
('S001','2025-04-16',124,3070),
('S002','2025-04-17',113,3170),
('S003','2025-04-18',137,2970),
('S004','2025-04-19',130,3140),
('S005','2025-04-21',115,2840),
('S002','2025-04-22',113,3170),
('S003','2025-04-22',137,2970),
('S004','2025-04-22',130,3140),
('S005','2025-04-22',115,2840);

-- Insert network_availability
INSERT INTO network_availability VALUES
('S001','2025-04-01',99.5),
('S002','2025-04-02',98.2),
('S003','2025-04-03',97.8),
('S004','2025-04-04',96.4),
('S005','2025-04-05',98.9),
('S001','2025-04-06',99.1),
('S002','2025-04-07',98.7),
('S003','2025-04-08',97.2),
('S004','2025-04-09',96.8),
('S005','2025-04-10',99.0),
('S001','2025-04-11',99.3),
('S002','2025-04-12',98.6),
('S003','2025-04-13',97.4),
('S004','2025-04-14',96.7),
('S005','2025-04-15',99.1),
('S001','2025-04-16',99.4),
('S002','2025-04-17',98.3),
('S003','2025-04-18',97.1),
('S004','2025-04-19',96.9),
('S005','2025-04-20',99.2),
('S001','2025-04-21',99.4),
('S002','2025-04-22',98.3),
('S003','2025-04-22',97.1),
('S004','2025-04-22',96.9);

-- Insert kpi_summary
INSERT INTO kpi_summary VALUES
('2025-04-01',4,2,3,98.5),
('2025-04-02',3,1,2,98.2),
('2025-04-03',5,3,1,97.8),
('2025-04-04',6,2,3,96.4),
('2025-04-05',2,2,2,98.9),
('2025-04-06',3,1,1,99.1),
('2025-04-07',4,2,3,98.7),
('2025-04-08',5,3,1,97.2),
('2025-04-09',6,2,3,96.8),
('2025-04-10',2,1,2,99.0),
('2025-04-11',3,1,1,99.3),
('2025-04-12',4,2,3,98.6),
('2025-04-13',5,3,1,97.4),
('2025-04-14',6,2,3,96.7),
('2025-04-15',2,2,2,99.1),
('2025-04-16',3,1,1,99.4),
('2025-04-17',4,2,3,98.3),
('2025-04-18',5,3,1,97.1),
('2025-04-19',6,2,3,96.9),
('2025-04-20',2,1,2,99.2),
('2025-04-22',3,1,1,94.4),
('2025-04-22',4,2,3,94.3),
('2025-04-22',5,3,1,92.1),
('2025-04-22',6,2,3,91.9);
""")

conn.commit()
print("DB is created with sample data")


DB is created with sample data


### Lets Check our data samples

In [12]:
conn = sqlite3.connect("my_data.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("\n📋 Tables in 'my_data.db':")
for table in tables:
    print(f" - {table[0]}")


📋 Tables in 'my_data.db':
 - alarm_data
 - incident_data
 - work_order
 - network_traffic
 - network_availability
 - kpi_summary


In [13]:
# Show sample data from each table
print("\n🔍 Sample data from each table:\n")
for (table_name,) in tables:
    print(f"--- {table_name.upper()} ---")
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 2;")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    print()


🔍 Sample data from each table:

--- ALARM_DATA ---
('A101', 'S001', 'Power Failure', 'Critical', '2025-04-01')
('A102', 'S001', 'Link Down', 'Major', '2025-04-02')

--- INCIDENT_DATA ---
('I201', 'S001', 'Link Failure', 'Closed', '2025-04-01', '2025-04-01')
('I202', 'S002', 'Power Issue', 'Open', '2025-04-02', '2025-04-03')

--- WORK_ORDER ---
('WO101', 'S001', 'Repair', 'Closed', '2025-04-01', '2025-04-02')
('WO102', 'S002', 'Upgrade', 'Open', '2025-04-02', '2025-04-03')

--- NETWORK_TRAFFIC ---
('S001', '2025-04-01', 120, 3000)
('S002', '2025-04-02', 110, 3200)

--- NETWORK_AVAILABILITY ---
('S001', '2025-04-01', 99.5)
('S002', '2025-04-02', 98.2)

--- KPI_SUMMARY ---
('2025-04-01', 4, 2, 3, 98.5)
('2025-04-02', 3, 1, 2, 98.2)



We now have our db ready lets move ahead

In [14]:
# Create and connect to SQlite DB
conn=sqlite3.connect("my_data.db")
cursor=conn.cursor()

In [19]:
# Extract Schema -- We will use this for Prompt
def extract_schema(conn):
    cursor=conn.cursor()
    tables=cursor.execute("Select name from sqlite_master Where type='table';").fetchall()
    schema={}
    
    for (table,) in tables :
        columns=cursor.execute(f"PRAGMA table_info ({table});").fetchall()
        schema[table]=[(col[1],col[2]) for col in columns]
    return schema
    
    

schema=extract_schema(conn)
schema
        

{'alarm_data': [('alarm_id', 'TEXT'),
  ('site_id', 'TEXT'),
  ('alarm_type', 'TEXT'),
  ('severity', 'TEXT'),
  ('timestamp', 'TEXT')],
 'incident_data': [('incident_id', 'TEXT'),
  ('site_id', 'TEXT'),
  ('incident_type', 'TEXT'),
  ('status', 'TEXT'),
  ('opened_at', 'TEXT'),
  ('closed_at', 'TEXT')],
 'work_order': [('wo_id', 'TEXT'),
  ('site_id', 'TEXT'),
  ('work_type', 'TEXT'),
  ('status', 'TEXT'),
  ('created_at', 'TEXT'),
  ('closed_at', 'TEXT')],
 'network_traffic': [('site_id', 'TEXT'),
  ('date', 'TEXT'),
  ('voice_traffic', 'INTEGER'),
  ('data_traffic', 'INTEGER')],
 'network_availability': [('site_id', 'TEXT'),
  ('date', 'TEXT'),
  ('availability_percent', 'REAL')],
 'kpi_summary': [('date', 'TEXT'),
  ('total_alarms', 'INTEGER'),
  ('total_incidents', 'INTEGER'),
  ('total_wos', 'INTEGER'),
  ('avg_availability', 'REAL')]}

### Lets add table description

In [20]:
table_description = {
    "alarm_data": "Stores alarm events from network nodes, including type, severity, and timestamps.",
    "incident_data": "Tracks reported incidents related to site issues, including type, status, and lifecycle timestamps.",
    "work_order": "Contains records of scheduled or reactive work orders issued to maintain or fix site infrastructure.",
    "network_traffic": "Logs daily traffic metrics for each site including voice and data traffic volumes.",
    "network_availability": "Monitors and stores the percentage of time each site is operational and available.",
    "kpi_summary": "Aggregates daily performance metrics like total alarms, incidents, work orders, and average availability."
}

In [23]:
#format for Prompt

def format_schema_with_description(schema_dict,description_dict):
    formatted = "" 
    for table,columns in schema_dict.items():
        formatted += f"Table: {table}\n"
        formatted += f"Description: {description_dict.get(table,'No Description Available')}\n"
        formatted += f"Columns:\n"
        for col_name,col_type in columns:
            formatted += f"  - {col_name }({col_type})\n"
        formatted += "\n"
    return formatted
schema_prompt=format_schema_with_description(schema,table_description)
schema_prompt
        

'Table: alarm_data\nDescription: Stores alarm events from network nodes, including type, severity, and timestamps.\nColumns:\n  - alarm_id(TEXT)\n  - site_id(TEXT)\n  - alarm_type(TEXT)\n  - severity(TEXT)\n  - timestamp(TEXT)\n\nTable: incident_data\nDescription: Tracks reported incidents related to site issues, including type, status, and lifecycle timestamps.\nColumns:\n  - incident_id(TEXT)\n  - site_id(TEXT)\n  - incident_type(TEXT)\n  - status(TEXT)\n  - opened_at(TEXT)\n  - closed_at(TEXT)\n\nTable: work_order\nDescription: Contains records of scheduled or reactive work orders issued to maintain or fix site infrastructure.\nColumns:\n  - wo_id(TEXT)\n  - site_id(TEXT)\n  - work_type(TEXT)\n  - status(TEXT)\n  - created_at(TEXT)\n  - closed_at(TEXT)\n\nTable: network_traffic\nDescription: Logs daily traffic metrics for each site including voice and data traffic volumes.\nColumns:\n  - site_id(TEXT)\n  - date(TEXT)\n  - voice_traffic(INTEGER)\n  - data_traffic(INTEGER)\n\nTable: n

### Combine few shot + Embedding+ FAISS

In [24]:
# Few Shot Examples
examples = [
    {
        "question": "How many alarms were raised today?",
        "sql": "SELECT COUNT(*) FROM alarm_data WHERE DATE(timestamp) = DATE('now');"
    },
    {
        "question": "How many number of incidents in the last three days?",
        "sql": "SELECT COUNT(*) FROM incident_data WHERE DATE(opened_at) >= DATE('now', '-3 days');"
    },
    {
        "question": "What is today's network availability?",
        "sql": "SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) = DATE('now');"
    },
    {
        "question": "How many work orders are currently open?",
        "sql": "SELECT COUNT(*) FROM work_order WHERE status = 'Open';"
    },
    {
        "question": "Show daily voice and data traffic for the past week.",
        "sql": "SELECT date, SUM(voice_traffic) AS total_voice, SUM(data_traffic) AS total_data FROM network_traffic WHERE DATE(date) >= DATE('now', '-7 days') GROUP BY date ORDER BY date;"
    },
    {
        "question": "Show number of alarms by severity for the past 7 days.",
        "sql": "SELECT severity, COUNT(*) FROM alarm_data WHERE DATE(timestamp) >= DATE('now', '-7 days') GROUP BY severity;"
    },
    {
        "question": "How was the network performance yesterday compared to the last 10 days?",
        "sql": """
        SELECT
            (SELECT AVG(avg_availability) FROM kpi_summary WHERE DATE(date) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS past_10_day_avg,
            (SELECT avg_availability FROM kpi_summary WHERE DATE(date) = DATE('now', '-1 day')) AS yesterday;
        """
    },
    {
        "question": "What is the trend of incidents over the last 14 days?",
        "sql": "SELECT DATE(opened_at) AS day, COUNT(*) AS incident_count FROM incident_data WHERE DATE(opened_at) >= DATE('now', '-14 days') GROUP BY day ORDER BY day;"
    },
    {
        "question": "How many incidents are open across all sites?",
        "sql": "SELECT COUNT(*) FROM incident_data WHERE status = 'Open';"
    },
    {
        "question": "List sites with the lowest availability in the last 5 days.",
        "sql": "SELECT site_id, AVG(availability_percent) AS avg_availability FROM network_availability WHERE DATE(date) >= DATE('now', '-5 days') GROUP BY site_id ORDER BY avg_availability ASC LIMIT 5;"
    },
    {
        "question": "How was the day yesterday?",
        "sql": """
        SELECT
            (SELECT COUNT(*) FROM alarm_data WHERE DATE(timestamp) = DATE('now', '-1 day')) AS alarms_yesterday,
            (SELECT COUNT(*) FROM incident_data WHERE DATE(opened_at) = DATE('now', '-1 day')) AS incidents_yesterday,
            (SELECT COUNT(*) FROM work_order WHERE DATE(created_at) = DATE('now', '-1 day')) AS work_orders_yesterday,
            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) = DATE('now', '-1 day')) AS availability_yesterday,
            
            (SELECT COUNT(*) * 1.0 / 10 FROM alarm_data WHERE DATE(timestamp) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS avg_alarms_last_10,
            (SELECT COUNT(*) * 1.0 / 10 FROM incident_data WHERE DATE(opened_at) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS avg_incidents_last_10,
            (SELECT COUNT(*) * 1.0 / 10 FROM work_order WHERE DATE(created_at) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS avg_work_orders_last_10,
            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS avg_availability_last_10;
        """
    },
    {
        "question": "How was the performance today?",
        "sql": """
        SELECT
            (SELECT COUNT(*) FROM alarm_data WHERE DATE(timestamp) = DATE('now')) AS alarms_today,
            (SELECT COUNT(*) FROM incident_data WHERE DATE(opened_at) = DATE('now')) AS incidents_today,
            (SELECT COUNT(*) FROM work_order WHERE DATE(created_at) = DATE('now')) AS work_orders_today,
            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) = DATE('now')) AS availability_today,

            (SELECT COUNT(*) * 1.0 / 10 FROM alarm_data WHERE DATE(timestamp) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_alarms_last_10,
            (SELECT COUNT(*) * 1.0 / 10 FROM incident_data WHERE DATE(opened_at) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_incidents_last_10,
            (SELECT COUNT(*) * 1.0 / 10 FROM work_order WHERE DATE(created_at) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_work_orders_last_10,
            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_availability_last_10;
        """
    },
    {
        "question": "Compare today's metrics with the last 10-day average.",
        "sql": """
        SELECT
            -- Today's counts
            (SELECT COUNT(*) FROM alarm_data WHERE DATE(timestamp) = DATE('now')) AS today_alarms,
            (SELECT COUNT(*) FROM incident_data WHERE DATE(opened_at) = DATE('now')) AS today_incidents,
            (SELECT COUNT(*) FROM work_order WHERE DATE(created_at) = DATE('now')) AS today_work_orders,
            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) = DATE('now')) AS today_availability,

            -- 10-day averages
            (SELECT COUNT(*) * 1.0 / 10 FROM alarm_data WHERE DATE(timestamp) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_alarms,
            (SELECT COUNT(*) * 1.0 / 10 FROM incident_data WHERE DATE(opened_at) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_incidents,
            (SELECT COUNT(*) * 1.0 / 10 FROM work_order WHERE DATE(created_at) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_work_orders,
            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_availability;
        """
    }
]


In [27]:
# COnvert to Langchain Documents
from langchain.docstore.document import Document
docs=[Document(page_content = ex["question"]+ " ||| " + ex["sql"]) for ex in examples]
docs

[Document(metadata={}, page_content="How many alarms were raised today? ||| SELECT COUNT(*) FROM alarm_data WHERE DATE(timestamp) = DATE('now');"),
 Document(metadata={}, page_content="How many number of incidents in the last three days? ||| SELECT COUNT(*) FROM incident_data WHERE DATE(opened_at) >= DATE('now', '-3 days');"),
 Document(metadata={}, page_content="What is today's network availability? ||| SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) = DATE('now');"),
 Document(metadata={}, page_content="How many work orders are currently open? ||| SELECT COUNT(*) FROM work_order WHERE status = 'Open';"),
 Document(metadata={}, page_content="Show daily voice and data traffic for the past week. ||| SELECT date, SUM(voice_traffic) AS total_voice, SUM(data_traffic) AS total_data FROM network_traffic WHERE DATE(date) >= DATE('now', '-7 days') GROUP BY date ORDER BY date;"),
 Document(metadata={}, page_content="Show number of alarms by severity for the past 7 da

In [ ]:
open_API_Key="your API Key"

In [29]:
# Initialize embedding model
embedding_model=OpenAIEmbeddings(openai_api_key=open_API_Key, model="text-embedding-3-small")

C:\Users\Comp\AppData\Local\Temp\ipykernel_11856\1411790866.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model=OpenAIEmbeddings(openai_api_key=open_API_Key, model="text-embedding-3-small")


In [30]:
#build FAISS vector index
vectorstore=FAISS.from_documents(docs,embedding_model)
vectorstore.save_local("faiss_index_mydata")

In [31]:
# Load Faiss and embedding index

vectorstore=FAISS.load_local(
    folder_path='faiss_index_mydata',
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

In [53]:
user_question="how was the day today"

# Retieve top k similar examples based on user's question

In [54]:
retrieved_docs=vectorstore.similarity_search(user_question,k=1)

In [55]:
retrieved_docs

[Document(metadata={}, page_content="How was the day yesterday? ||| \n        SELECT\n            (SELECT COUNT(*) FROM alarm_data WHERE DATE(timestamp) = DATE('now', '-1 day')) AS alarms_yesterday,\n            (SELECT COUNT(*) FROM incident_data WHERE DATE(opened_at) = DATE('now', '-1 day')) AS incidents_yesterday,\n            (SELECT COUNT(*) FROM work_order WHERE DATE(created_at) = DATE('now', '-1 day')) AS work_orders_yesterday,\n            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) = DATE('now', '-1 day')) AS availability_yesterday,\n            \n            (SELECT COUNT(*) * 1.0 / 10 FROM alarm_data WHERE DATE(timestamp) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS avg_alarms_last_10,\n            (SELECT COUNT(*) * 1.0 / 10 FROM incident_data WHERE DATE(opened_at) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS avg_incidents_last_10,\n            (SELECT COUNT(*) * 1.0 / 10 FROM work_order WHERE DATE(created

In [56]:
#Parse them back to structured examples
examples=[]
for doc in retrieved_docs :
    q_sql=doc.page_content.split(" ||| ")
    examples.append({"question": q_sql[0].strip(),
                     "sql": q_sql[1].strip()
                     })
    
examples

[{'question': 'How was the day yesterday?',
  'sql': "SELECT\n            (SELECT COUNT(*) FROM alarm_data WHERE DATE(timestamp) = DATE('now', '-1 day')) AS alarms_yesterday,\n            (SELECT COUNT(*) FROM incident_data WHERE DATE(opened_at) = DATE('now', '-1 day')) AS incidents_yesterday,\n            (SELECT COUNT(*) FROM work_order WHERE DATE(created_at) = DATE('now', '-1 day')) AS work_orders_yesterday,\n            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) = DATE('now', '-1 day')) AS availability_yesterday,\n            \n            (SELECT COUNT(*) * 1.0 / 10 FROM alarm_data WHERE DATE(timestamp) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS avg_alarms_last_10,\n            (SELECT COUNT(*) * 1.0 / 10 FROM incident_data WHERE DATE(opened_at) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS avg_incidents_last_10,\n            (SELECT COUNT(*) * 1.0 / 10 FROM work_order WHERE DATE(created_at) BETWEEN DATE('now',

In [57]:
# few shot prompt

example_prompt=PromptTemplate(
    input_variables=["question","sql"],
    template="Question: {question}\n SQL: {sql}"
    
)
example_prompt


PromptTemplate(input_variables=['question', 'sql'], input_types={}, partial_variables={}, template='Question: {question}\n SQL: {sql}')

In [58]:
prompt=FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="""
    Behave like a SQL query writer. Use the SQL Schema and descriptions below :
    {schema}
    
    Now based on the user's question : "{question}", write an accurate SQL query. 
    Do not try to reduce the SQL Query if it gets complex.
    """,
    input_variables =["schema","question"]
)
prompt

FewShotPromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, examples=[{'question': 'How was the day yesterday?', 'sql': "SELECT\n            (SELECT COUNT(*) FROM alarm_data WHERE DATE(timestamp) = DATE('now', '-1 day')) AS alarms_yesterday,\n            (SELECT COUNT(*) FROM incident_data WHERE DATE(opened_at) = DATE('now', '-1 day')) AS incidents_yesterday,\n            (SELECT COUNT(*) FROM work_order WHERE DATE(created_at) = DATE('now', '-1 day')) AS work_orders_yesterday,\n            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) = DATE('now', '-1 day')) AS availability_yesterday,\n            \n            (SELECT COUNT(*) * 1.0 / 10 FROM alarm_data WHERE DATE(timestamp) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS avg_alarms_last_10,\n            (SELECT COUNT(*) * 1.0 / 10 FROM incident_data WHERE DATE(opened_at) BETWEEN DATE('now', '-11 days') AND DATE('now', '-2 days')) AS avg_incidents_l

# Define the llm chain

In [59]:
llm=ChatOpenAI(openai_api_key=open_API_Key,model_name='gpt-4')

In [60]:
sql_chain=LLMChain(llm=llm,prompt=prompt)

In [61]:
#generate SQL Query now
sql_query=sql_chain.run(schema=schema_prompt,question=user_question)
sql_query

"SQL: SELECT\n            (SELECT COUNT(*) FROM alarm_data WHERE DATE(timestamp) = DATE('now')) AS alarms_today,\n            (SELECT COUNT(*) FROM incident_data WHERE DATE(opened_at) = DATE('now')) AS incidents_today,\n            (SELECT COUNT(*) FROM work_order WHERE DATE(created_at) = DATE('now')) AS work_orders_today,\n            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) = DATE('now')) AS availability_today,\n            \n            (SELECT COUNT(*) * 1.0 / 10 FROM alarm_data WHERE DATE(timestamp) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_alarms_last_10,\n            (SELECT COUNT(*) * 1.0 / 10 FROM incident_data WHERE DATE(opened_at) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_incidents_last_10,\n            (SELECT COUNT(*) * 1.0 / 10 FROM work_order WHERE DATE(created_at) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_work_orders_last_10,\n            (SELECT AVG(availability_

In [62]:
# Remove Accidental Prefix
if sql_query.strip().startswith("SQL:"):
    sql_query=sql_query.split("SQL:",1)[1].strip()
sql_query

"SELECT\n            (SELECT COUNT(*) FROM alarm_data WHERE DATE(timestamp) = DATE('now')) AS alarms_today,\n            (SELECT COUNT(*) FROM incident_data WHERE DATE(opened_at) = DATE('now')) AS incidents_today,\n            (SELECT COUNT(*) FROM work_order WHERE DATE(created_at) = DATE('now')) AS work_orders_today,\n            (SELECT AVG(availability_percent) FROM network_availability WHERE DATE(date) = DATE('now')) AS availability_today,\n            \n            (SELECT COUNT(*) * 1.0 / 10 FROM alarm_data WHERE DATE(timestamp) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_alarms_last_10,\n            (SELECT COUNT(*) * 1.0 / 10 FROM incident_data WHERE DATE(opened_at) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_incidents_last_10,\n            (SELECT COUNT(*) * 1.0 / 10 FROM work_order WHERE DATE(created_at) BETWEEN DATE('now', '-10 days') AND DATE('now', '-1 day')) AS avg_work_orders_last_10,\n            (SELECT AVG(availability_perce

### Code for SQL Execution

In [63]:
conn=sqlite3.connect("my_data.db")
def execute_sql_query(sql_query):
    df=pd.read_sql_query(sql_query,conn)
    
    return df
    

In [64]:
df=execute_sql_query(sql_query)
df

,alarms_today,incidents_today,work_orders_today,availability_today,avg_alarms_last_10,avg_incidents_last_10,avg_work_orders_last_10,avg_availability_last_10
0,2,2,2,97.433333,1.0,0.7,0.7,98.21


Classify Question type for Output

In [65]:
def classify_question_type(llm,question):
    classification_prompt = f""" 
    classify the following user question into one of the following category:
    - "daily_summary" : Question asking for overall network status for today vs past. 
    - "trend" : Question asking for data trends over time (line charts). 
    - " table" : Question asking for raw tabuler data (top alarms, work orders etc). 
    - "compare_period" : Comparing two different time periods. (e.g. last week vs this week). 
    - "top_n" : Asking for top or worst entities (e.g. top 5 failing nodes). 
    
    Question : "{question}" 
    Answer (just the label):
    
    """ 
    response= llm.predict(classification_prompt)
    return response.strip().lower()
        
    

In [66]:
question_type=classify_question_type(llm,user_question)

C:\Users\Comp\AppData\Local\Temp\ipykernel_11856\2768668067.py:14: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response= llm.predict(classification_prompt)


In [67]:
question_type

'"daily_summary"'

In [70]:
if question_type =='"daily_summary"' :
    today_vs_avg ={}
    score=0
    for col in df.columns:
        if 'today' in col:
            metric_name=col.replace('today_','')
            today_val=df[col].iloc[0]
            avg_col=f'avg_{metric_name}_7d'
            if avg_col in df.columns:
                avg_val=df[avg_col].iloc[0]
                today_vs_avg[metric_name]=(today_val,avg_val)
    for metric,(today,avg)in today_vs_avg.items():
        if metric in ['alarms','incidents','wos']:
            if today<avg *0.8:
                score+=1
            elif today> avg*1.2 :
                score -= 1
        elif metric in ['availability' ,'kpi']:
            if today> avg*1.05 :
                score += 1
            elif today < avg*0.95 :
                score -=1
                
    if score >= 2 : 
        print( "it was a ** Good Day ** for the network!")
    elif score <= -2 :
        print( "it was a ** Bad Day ** for the network!")
    else :
        print( "it was a ** Normal Day ** for the network!")
        
    print (" here is the summary :")
    for col in df.columns :
        print (f"{col.replace('_',' ').capitalize()} : {df[col].iloc[0]}")
elif question_type =='"trend"' :
    df.plot(x=df.columns[0],y=df.columns[1:],kind='line',marker='o')
    plt.title("Trend")
    plt.xlabel("Date")
    plt.ylable("Values")
    plt.tight_layout()
    plt.show()
elif question_type=='"top_n"':
    print ("\n Top-N Entities :")
    print(df)
else :
    print ("\n Raw Table Output :")
    print(df)

it was a ** Normal Day ** for the network!
 here is the summary :
Alarms today : 2
Incidents today : 2
Work orders today : 2
Availability today : 97.43333333333332
Avg alarms last 10 : 1.0
Avg incidents last 10 : 0.7
Avg work orders last 10 : 0.7
Avg availability last 10 : 98.21
